In [3]:
import requests
from bs4 import BeautifulSoup
import datetime
import time
import threading
import queue

In [4]:
%%capture
pip install --upgrade firebase-admin 


In [5]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore

if not firebase_admin._apps:
    cred = credentials.Certificate("jsonFile/service-account.json")
    firebase_admin.initialize_app(cred)
    
db = firestore.client()

In [6]:

def currentTime():
    now = datetime.datetime.now()
    currentTime = now.strftime("%Y-%m-%d %H:%M:%S")
    return currentTime

In [7]:
#personal detail
def personalData(soup):
    info = {}
    content = ''.join(soup.find('meta', property = 'og:image')['content']) #list to string
    id = content.split('user=')[1].split('&citpid')[0] 

    d = soup.find('div', id = 'gsc_prf_i')

    #name
    info['name'] = d.find('div', id = 'gsc_prf_in').text

    #university
    info['university'] = d.find('a', class_ = 'gsc_prf_ila').text

    #email
    email = d.find('div', id = 'gsc_prf_ivh').text.split(' ')[1]
    info['email'] = email

    #picture
    info['picture'] = soup.find('div', id = 'gsc_prf_pua').find('img')['src'] 
    
    label = []
    for p in soup.find_all('a', class_='gsc_prf_inta gs_ibl'):

        label.append(p.text)
    info['label'] = label
    
    return info


In [8]:
def citeBy(soup):
    citeBy = {}
    citations = {}
    h_index = {}
    i10_index = {}
    def cited(status, value):

        if status / 2 < 1:        
            if status % 2 == 0:
                citations['All'] = value
            else:
                citations['Since2016'] = value
            citeBy['citations'] = citations

        if status / 2 < 2:
            if status % 2 == 0:
                h_index['All'] = value
            else:
                h_index['Since2016'] = value
            citeBy['h_index'] = h_index

        if status / 2 < 3:
            if status % 2 == 0:
                i10_index['All'] = value
            else:
                i10_index['Since2016'] = value
            citeBy['i10_index'] = i10_index
            
    count_d = 0
    for d in soup.find_all('td', class_='gsc_rsb_std'):

        cited(count_d, d.text)
        count_d = count_d + 1       
    
    return citeBy

In [9]:
def result(soup, ID):
    infos = {}
    infos['id'] = ID
    infos['updateTime'] = currentTime()
    infos['personalData'] = personalData(soup)
    infos['cited'] = citeBy(soup)

    return infos

In [10]:
def personalPage(id):
    url = 'https://scholar.google.com.tw/citations?hl=zh-TW&user=' + id
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    return result(soup, id)

In [11]:
# import time
# import threading
# import queue
def userDataList(limit):
    userDataList = []
    # 建立佇列
    ID_queue = queue.Queue()

    # 將資料放入佇列
    users_ref = db.collection(u'cguscholar').limit(limit)
    docs = users_ref.get()

    for doc in docs:
        #print(u'{}'.format(doc.id))
        ID_queue.put((doc.id))

    # Worker 類別，負責處理資料
    class Worker(threading.Thread):
      def __init__(self, queue, num):
        threading.Thread.__init__(self)
        self.queue = queue
        self.num = num

      def run(self):
        while self.queue.qsize() > 0:
          # 取得新的資料
          userID = self.queue.get()

          # 處理資料
          #print("Worker %d: %s" % (self.num, userID))
          userDataList.append(personalPage(userID))

          time.sleep(1)

    # 建立兩個 Worker
    my_worker1 = Worker(ID_queue, 1)
    my_worker2 = Worker(ID_queue, 2)

    # 讓 Worker 開始處理資料
    my_worker1.start()
    my_worker2.start()

    # 等待所有 Worker 結束
    my_worker1.join()
    my_worker2.join()

    print("Done.")
    return userDataList


In [56]:
import json
data = json.dumps(userDataList(3))
jsonStr = json.loads(data) 
print(jsonStr)


Done.
[{'id': 'bjEpXBoAAAAJ', 'updateTime': '2021-09-20 13:44:54', 'personalData': {'name': 'Matti Pietikäinen', 'university': 'University of Oulu', 'email': 'oulu.fi', 'picture': 'https://scholar.googleusercontent.com/citations?view_op=view_photo&user=bjEpXBoAAAAJ&citpid=2', 'label': ['computer vision', 'pattern recognition', 'biometrics', 'affective computing', 'artificial intelligence']}, 'cited': {'citations': {'All': '73213', 'Since2016': '37876'}, 'h_index': {'All': '94', 'Since2016': '70'}, 'i10_index': {'All': '261', 'Since2016': '178'}}}, {'id': 'faE3_ksAAAAJ', 'updateTime': '2021-09-20 13:44:54', 'personalData': {'name': 'Jeffrey M. Wooldridge', 'university': 'Michigan State University', 'email': 'msu.edu', 'picture': 'https://scholar.googleusercontent.com/citations?view_op=view_photo&user=faE3_ksAAAAJ&citpid=2', 'label': ['econometrics', 'panel data', 'treatment effects', 'clustering', 'economics']}, 'cited': {'citations': {'All': '118631', 'Since2016': '54884'}, 'h_index': 

In [57]:
def updatetest(items):
    count_item = 0
    for i in items:
        ref = db.collection(u'cguscholar').document((items[count_item]['id']))
        ref.collection(u'updateTime').document((items[count_item]['updateTime'])).set(items[count_item]['cited'])
        ref.set(items[count_item]['personalData'])
        count_item = count_item + 1

In [58]:
updatetest(jsonStr)

In [88]:
jsonFile = open("jsonFile/personalData.json", "w")
jsonFile.write(jsonStr)
jsonFile.close()

In [41]:
#firebase connect test

In [93]:
import json
 
# Opening JSON file
f = open('jsonFile/personalData.json',)
 
# returns JSON object as
# a dictionary
data = json.load(f)



JSONDecodeError: Expecting value: line 1 column 1 (char 0)